In [1]:
import sys
sys.path.append("..")

from Agents import *
from mtg_etl_pipeline import *

## THE NEW WAY: USING THE ETL MODULE
This process will do the exact same thing as you will see in the "OLD WAY" demo, just in a cleaner way. Rememeber that you'll need selenium + chromedriver downloaded for this to work. With the 3 code cells below, you can create/validate all the inputs you'll need for an experiment in less than 2 minutes. 

In [5]:
##PART 1: PULL DATA FROM 17LANDS -> Source_Weights File
#Instantiate the client we will use to pull data and create source df from
mtg_etl_client_instance = mtg_etl_client(abs_path_to_dir="C:/Users/trist/OneDrive/Desktop/drafting-research/",url='https://www.17lands.com/card_ratings', 
archetypes=['WU','WB','WR','WG','UB','UR','UG','BR','BG','RG'], 
expansion='VOW',
headless=True)

#Here you put in your path to the chromedriver .exe file
#The raw df gets stored on the client, so we don't need to worry about the raw html pages
mtg_etl_client_instance.pull_raw_card_rankings('C:/Users/trist/Downloads/chromedriver.exe')

#Now we parse the data for a given set of columns
raw_df = mtg_etl_client_instance.parse_html_to_raw_df(['# Seen', '# GP'])

##OPTIONAL
#If you want to add complex logic for imputing (e.g. use some value when under X number of picks, you would add those columns above
# and write some custom logic at this step to get what you want).
new_col_df = mtg_etl_client_instance.generate_computed_column_from_raw_targets(raw_df, '# GP', '# Seen', "gp_over_seen", "div")

#Send to source_weights 
mtg_etl_client_instance.process_raw_client_df_to_source_csv(new_col_df,'test')

In [3]:
##PART 2: TRANSFORMING SOURCE DATA, VALIDATING TRANSFORMED DATA, WRITING TO CSV/JSON FOR EXPERIMENTATION
data_processor = mtg_processed_csv_json_generator("C:/Users/trist/OneDrive/Desktop/drafting-research/",'VOW_Weights_default_test_df.csv', 'gp_over_seen_')
data_processor.convert_source_df_to_processed_df(min_max_scale=True,min_max_range=(1,5))
data_processor.write_to_processed_csv()
data_processor.write_to_json_weights_writeback()

In [3]:
##PART 3: CONVERTING MTGJSON RAW FILES INTO INPUTS FOR EQUILIBRIUM EXPERIMENTS
#note this is only needed once per set that you'd like to investigate
mtg_json_path = "json_files/VOW.json"
default_weight_df_path = "weights_data/source_weights/VOW_Weights_default_test_df.csv"

set_writeback_processor = mtg_set_writeback_generator(mtg_json_path,
    default_weight_df_path,
    expansion="VOW",
    abs_path_to_dir="C:/Users/trist/OneDrive/Desktop/drafting-research/")

#The workflow here is to generate a writeback, make sure that there are no missing/corrupted cards in what we are writing,
#and if we have the all clear, we can generate a writeback (the final function has a failsafe so it won't write if validation has
# not been successful)
set_writeback_processor.generate_writeback()
set_writeback_processor.validate_writeback()

#There's an argument for a prefix if you want, but you can also just treat it as a blank string
set_writeback_processor.writeback_to_json('')

Adamant Will
added normally
Angelic Quartermaster
added normally
Arm the Cathars
added normally
Bride's Gown
added normally
By Invitation Only
added normally
Cemetery Protector
added normally
Circle of Confinement
added normally
Dawnhart Geist
added normally
Distracting Geist
added
Distracting Geist
passed
Drogskol Infantry
added
Drogskol Infantry
passed
Estwald Shieldbasher
added normally
Faithbound Judge
added
Faithbound Judge
passed
Fierce Retribution
added normally
Fleeting Spirit
added normally
Gryff Rider
added normally
Gryffwing Cavalry
added normally
Hallowed Haunting
added normally
Heron of Hope
added normally
Heron-Blessed Geist
added normally
Hopeful Initiate
added normally
Katilda, Dawnhart Martyr
added
Katilda, Dawnhart Martyr
passed
Kindly Ancestor
added
Kindly Ancestor
passed
Lantern Flare
added normally
Militia Rallier
added normally
Nebelgast Beguiler
added normally
Nurturing Presence
added normally
Ollenbock Escort
added normally
Panicked Bystander
added
Panicked Byst

## THE OLD WAY: Beautiful Soup + Selenium to get new weights from 17lands
Note, this process does not even cover doing JSON writebacks because that was ad-hoc when this was first made. (It covers most of #1 above and that's about it)

In [ ]:
# from selenium import webdriver 
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from bs4 import BeautifulSoup
# import time

# chrome_options = Options()

# chrome_options.add_argument("--headless")

# archetypes = ['WU','WB','WR','WG','UB','UR','UG','BR','BG','RG']

# expansion_string = 'VOW'


# #store the url of the first page
# url = 'https://www.17lands.com/card_ratings'
# #define the executable path on the webdriver using the file's destination 
# driver = webdriver.Chrome(executable_path='C:/Users/trist/Downloads/chromedriver.exe',options=chrome_options)
# try:
#     #get the url in the driver, use either URL or URL_two depending on which page you're trying to scrape
#     driver.get(url)

#     #Now let's make sure that we are choosing the right set
#     select = Select(driver.find_element_by_id('expansion'))

#     # select by visible text
#     select.select_by_visible_text(expansion_string)

#     #Wait for our table body to load
#     table = WebDriverWait(driver, 20).until(EC.visibility_of_element_located(
#         (By.XPATH, '//tbody')))

#     #Grab the page source
#     html=driver.page_source

#     #5 second cooldown 
#     time.sleep(5)

#     #Create list of html tables that we will later iterate through
#     html_list = []
#     for arch in archetypes:
#         #Now let's make sure that we are choosing the right set
#         select = Select(driver.find_element_by_id('deck_color'))

#         # select by visible text
#         select.select_by_visible_text(arch)

#         #Wait for our table body to load
#         table = WebDriverWait(driver, 20).until(EC.visibility_of_element_located(
#         (By.XPATH, '//tbody')))

#         #Grab the page source
#         new_html=driver.page_source

#         html_list.append(new_html)

#         #5 second cooldown 
#         time.sleep(5)

#     driver.quit()
# except TimeoutError as tie:
#     #Error Handling 
#     print("Exception " +tie+ 'has occured')
#     driver.quit()


In [ ]:

# soup = BeautifulSoup(html,'html.parser')
# div = soup.select_one('table')
# t2 = pd.read_html(div.prettify())[0]

# for idx, tbl in enumerate(html_list):
#     soup = BeautifulSoup(tbl,'html.parser')
#     div = soup.select_one('table')
#     new_table = pd.read_html(div.prettify())[0].loc[:,['Name','# GP','GP WR']]
#     #new_table['IWD'] = np.where((new_table['# Picked']>1700)&(new_table['Rarity']!='M'),new_table['IWD'],0)
#     new_table['GP WR'] = np.where((new_table['# GP']>=800),new_table['GP WR'],0)
#     #new_table['Seen_rate'] = new_table['# GP'] /new_table['# Seen'] 
#     new_table = new_table.loc[:,['Name','GP WR']]
#     t2 = t2.merge(right=new_table,how='inner',on='Name',suffixes=[None," " + archetypes[idx]])

In [ ]:
# # #This cell actually does things.... here we will impute null values using the card's avg IWD
# impute_colnames = t2.iloc[:,-len(archetypes):].columns.values
# t2copy = t2.copy()

# # #Make all nulls np.nan so we can remove them with the min with a masked array
# # t2copy['IWD'] = t2copy['IWD'].fillna('999.9pp')

# # #Remove the pp suffix from our numbers and convert to float
# # t2copy['IWD'] = t2copy['IWD'].str.replace('pp','').astype('float')

# # #Create masked array that will look for all positions where we have np.nan and replace it with the min
# # #The intuition here is if we don't have enough data for the IWD of a card at all, people are against picking it all, thus meaning it must have a low IWD
# # masked_iwd = t2copy['IWD'].mask(
# #     t2copy['IWD'].eq(999.9),
# #     t2copy['IWD'].min())

# # #Replace old IWD column with our masked_iwd, which is a float
# # t2copy['IWD'] = masked_iwd


# for c in impute_colnames:

#     #Generally, we will fill the specialized cols with the overall IWD when the sample is small
#     t2copy[c] = t2copy[c].fillna(0).astype('string')

#     #For those we aren't replacing, we also need to clean off the 'pp' string and convert datatypes
#     t2copy[c] = t2copy[c].str.replace('%','').astype('float')


# tz = t2copy.iloc[:,-10:].columns.values
# tz = np.append(tz,['Name','Color'])
# tt = t2copy.loc[:,tz]


# #Now we deal with the weights of the missing lands. 
# # Since IWD refers to an increase in win percentage, it makes the most sense of the IWD of a land to be 0 
# #Concept here being unless it hurts your deck, you should take another card

# lands = ['Swamp', 'Forest', 'Island', 'Plains', 'Mountain']
# colors =['B','G','U','W','R']
# zeros = [0 for x in archetypes]

# output = []
# for idx, l in enumerate(lands):
#     new_zeros = zeros.copy()
#     new_zeros.append(l)
#     new_zeros.append(colors[idx])
#     output.append(new_zeros)

In [ ]:
# dummies_df = pd.DataFrame(output, columns=tt.columns.values)

# final_weights_df = pd.concat([tt, dummies_df]).sort_values(by='Name').reset_index(drop=True)

In [ ]:
# final_weights_df.shape[0]

In [ ]:
# final_weights_df = final_weights_df.loc[:, final_weights_df.columns != 'IWD']

In [ ]:
# final_weights_df

In [ ]:
# final_weights_df.to_csv('VOM_Weights_default__GPWR_df.csv', index=False)